In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sporboost.forest import *
import pandas as pd
from time import time
import os
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier

################################################################################
################################################################################
################################################################################
# Data Load
def load_simulated(path):
    data = np.loadtxt(path, delimiter=",")
    return data[:,:-1], data[:,-1].astype('int32')

dfs = {
    'iris' : load_iris(return_X_y = True),
    'breast_cancer' : load_breast_cancer(return_X_y = True),
    'wine' : load_wine(return_X_y = True),
    'trunk' : load_simulated("source_data/simulated/Trunk_train.csv"),
    'orthant' : load_simulated("source_data/simulated/Orthant_train.csv"),
    'sparse_parity' : load_simulated("source_data/simulated/Sparse_parity_train.csv")
}


C:\Users\Ed\anaconda3\envs\proj\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
################################################################################
################################################################################
################################################################################
# Hyperparameters across all estimators
# Boosted Forest
boosted_max_depth = (1,2)

# Sparse Random Trees
d_ratio = (0.5, 1)
s = (3, 4)

# Rotational Decision Trees
K = (2, 3)


In [3]:
################################################################################
################################################################################
################################################################################
# Scoring Functions
scoring = {
    "cohens_kappa" : make_scorer(cohen_kappa_score),
    "accuracy" : 'accuracy'
}

In [4]:
################################################################################
################################################################################
################################################################################
# Initalize cross-validation params
pipe = Pipeline([
    ('clf', RandomForest())
])

params = [
    {
        'clf' : (RandomForest(),),
    },
    {
        'clf' : (AdaBoost(),),
        'clf__max_depth' : boosted_max_depth
    },
    {
        'clf' : (SPORF(),),
        'clf__d_ratio' : d_ratio,
        'clf__s' : s
    },
    {
        'clf' : (SPORBoost(),),
        'clf__max_depth' : boosted_max_depth,
        'clf__d_ratio' : d_ratio,
        'clf__s' : s
    },
    {
        'clf' : (RotationalForest(),),
        'clf__K' : K
    },
    {
        'clf' : (RotBoost(),),
        'clf__max_depth' : boosted_max_depth,
        'clf__K' : K
    },
    {
        'clf' : (XGBClassifier(nthread = 1), )
    }
]

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=20220329)

cv_base = GridSearchCV(estimator = pipe,
                       param_grid = params,
                       cv=kfold,
                       n_jobs=-1,
                       return_train_score=True,
                       refit=False,
                       scoring=scoring
)


In [5]:
################################################################################
################################################################################
################################################################################
# Func to write results to disk
def write_results(cv, label):
    front_cols = ['param_clf',
                  'mean_test_cohens_kappa', 'std_test_cohens_kappa',
                  'mean_test_accuracy', 'std_test_accuracy',
                  'mean_train_cohens_kappa', 'std_train_cohens_kappa',
                  'mean_train_accuracy', 'std_train_accuracy',
                  'param_clf__max_depth',
                  'param_clf__d_ratio', 'param_clf__s',
                  'param_clf__K'
                ]

    results = pd.DataFrame(cv.cv_results_).drop(['params'], axis=1)
    results['param_clf'] = results['param_clf'].apply(lambda c: type(c).__name__)
    results = results.sort_values(['param_clf', 'mean_test_cohens_kappa'],
                                ascending=False, ignore_index = True)
            
    results = results[front_cols + results.drop(front_cols, axis=1).columns.tolist()]
    results.to_csv(f"report/{label}.csv", index=False)

In [6]:
################################################################################
################################################################################
################################################################################
# Fit the CVs
for name, df in dfs.items():
    if not os.path.exists(f"report/{name}.csv"):
        start = time()
        print(f"Testing {name}")
        write_results(cv_base.fit(*df), name)
        end = time()
        print(f"Finished {name} in {(end - start) / 60:.1f} mins")
    else:
        print(f"Skipping {name} as it has already been tested")

Testing iris
Finished iris in 1.3 mins
Skipping breast_cancer as it has already been tested
Skipping wine as it has already been tested
Skipping trunk as it has already been tested
Skipping orthant as it has already been tested
Skipping sparse_parity as it has already been tested
